When docker chokes on containers that have worked before, try clearing out disk space:
(from https://stackoverflow.com/questions/30719896/docker-dm-task-run-failed-error)

sudo service docker stop
sudo thin_check /var/lib/docker/devicemapper/devicemapper/metadata
sudo thin_check --clear-needs-check-flag /var/lib/docker/devicemapper/devicemapper/metadata
sudo service docker start


Imports and set some global variables

In [17]:
import shutil
import os
import subprocess
import sys
from datetime import datetime, timedelta
import glob
import ftplib
import tarfile
import os.path

base_dir = '/work/markstro/operat/setup/test/'
work_dir = base_dir + 'NHM-PRMS_CONUS'
ftp_server = 'ftpint.usgs.gov'
ftp_path = 'pub/cr/co/denver/BRR-CR/pub/markstro/onhm/'
ftp_login = 'anonymous'
ftp_pw = ''
python_module_path = "/work/markstro/operat/repos/onhm-runners/"

Create the NHM-PRMS workspace directory and subdirectories. This downloads param file, control file, etc. It creates the PRMS workspace for the ONHM.

In [9]:
# Remove the old work space
if os.path.exists(work_dir):
    shutil.rmtree(work_dir)
    
os.chdir(base_dir)

filename = 'NHM-PRMS_CONUS.tar.gz'

# Get the tar ball from ftp
ftp = ftplib.FTP(ftp_server) 
ftp.login(ftp_login, ftp_pw) 
ftp.cwd(ftp_path)
ftp.retrbinary("RETR " + filename, open(filename, 'wb').write)
ftp.quit()

# unzip and untar
tf = tarfile.open(base_dir + filename)
tf.extractall()

Install thredds server into NHM_PRMS workspace


Not implemented yet.

Make the historical PRMS run offline. There are still lots of issues (ie, updats to the HRUs and the current CBH files end in 2014), but goto

/work/markstro/operat/setup/init_file/NHM-PRMS_CONUS

to run "run_for_init.sh" that will create the prms_save_XXXX_XX_XX.init file.

Right now, download an init file from the ftp server to illustrate that the initial init file comes from outside of what is going on here.

In [10]:
os.chdir(work_dir)

filename = 'prms_save_2019-03-21.init'

# Get the init file from ftp
ftp = ftplib.FTP(ftp_server) 
ftp.login(ftp_login, ftp_pw) 
ftp.cwd(ftp_path)
ftp.retrbinary("RETR " + filename, open(filename, 'wb').write)
ftp.quit()

'221 Goodbye.'

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Here starts the operational loop. The following steps are repeated everyday.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Figure out the dates.

In [11]:
yesterday = datetime.today() - timedelta(1)
sixtydays = yesterday - timedelta(60)
initdate = sixtydays - timedelta(1)

model_start_date = sixtydays
model_end_date = yesterday

fetch_start_date = initdate
fetch_end_date = yesterday

# The init_start_date is one day after whatever the current init_date is. Get this from the
# file in the workspace.
os.chdir(work_dir)
for file in glob.glob("*.init"):
    ;
    
tok = file.split('_')
foo = tok[2]
tok = foo.split('.')
date_str = tok[0]

init_start_date = datetime.strptime(date_str, '%Y-%m-%d')
init_end_date = sixtydays

print('model_start_date = ' + model_start_date.strftime('%Y-%m-%d'))
print('model_end_date = ' + model_end_date.strftime('%Y-%m-%d'))
print('fetch_start_date = ' + fetch_start_date.strftime('%Y-%m-%d'))
print('fetch_end_date = ' + fetch_end_date.strftime('%Y-%m-%d'))
print('init_start_date = ' + init_start_date.strftime('%Y-%m-%d'))
print('init_end_date = ' + init_end_date.strftime('%Y-%m-%d'))


model_start_date = 2019-03-31
model_end_date = 2019-05-30
fetch_start_date = 2019-03-30
fetch_end_date = 2019-05-30
init_start_date = 2019-03-21
init_end_date = 2019-03-31


Run the Fetcher-Parser to bring the CBH netcdf up to yesterday. This copy from stage business simulates that Fetcher-Parser.

Not implemented yet, but pull from ftp as a place holder.

In [12]:
os.chdir(work_dir)

filename = 'new.nc'

# Get the init file from ftp
ftp = ftplib.FTP(ftp_server) 
ftp.login(ftp_login, ftp_pw) 
ftp.cwd(ftp_path)
ftp.retrbinary("RETR " + filename, open(filename, 'wb').write)
ftp.quit()

# shutil.copy(stage_dir + 'stage_2/new.nc', work_dir)

'221 Goodbye.'

Pull the container for the Fetcher-Parser

In [6]:
os.chdir(work_dir)

# docker pull for ofp
arg = 'docker pull nhmusgs/ofp'
arg_l = arg.split() # need this for whatever reason; don't move arg.split into the func call below

subprocess.check_output(arg_l)

CalledProcessError: Command '['docker', 'pull', 'nhmusgs/ofp']' returned non-zero exit status 1.

In [ ]:
# arg = 'docker run --rm -v ' + work_dir + ':/work nhmusgs/prms:5.0.0 ' + fa + sa + ea + ia
# arg_l = arg.split() # need this for whatever reason; don't move arg.split into the func call below

# subprocess.check_output(arg_l)

Run from onhm-runners ncf2cbh.ncf2cbh.py. This will write PRMS format CBH files that contain
data from 2019-01-01 up through yesterday.

It's here, but needs more work!

In [13]:
# Not sure about the best way to handle importation of modules in jupyter,
# but this seems to work
# My PRMS python utilities are in onhm-runners/prms_utils
if python_module_path not in sys.path:
    sys.path.append(python_module_path)

# Spell out full paths -- don't use string variables -- don't know why.
%run '/work/markstro/operat/repos/onhm-runners/ncf2cbh/ncf2cbh.py' '/work/markstro/operat/setup/test/NHM-PRMS_CONUS/'

setting dir = /work/markstro/operat/setup/test/NHM-PRMS_CONUS/
base_date_str 2019-05-13 00:00:00
2019-05-13
writing /work/markstro/operat/setup/test/NHM-PRMS_CONUS/prcp.cbh
writing /work/markstro/operat/setup/test/NHM-PRMS_CONUS/tmax.cbh
writing /work/markstro/operat/setup/test/NHM-PRMS_CONUS/tmin.cbh


Run PRMS-NHM in the workspace. This run should always go from the day after the current
init file up through yesterday.

The next step works to run PRMS, but the start_time and end_time logic is not figured out yet.

In [16]:
os.chdir(work_dir)
#arg = 'docker run --rm -v ' + work_dir + ':/work prms:5.0.0 ./NHM-PRMS.control'

fa = './NHM-PRMS.control -set init_vars_from_file 1 -set save_vars_to_file 0'

# figure out the following from start_date and end_date
sa = ' -set start_date ' + str(model_start_date.year) + ',' + str(model_start_date.month) + ',' + str(model_start_date.day) + ',0,0,0'
print(sa)
ea = ' -set end_date ' + str(model_end_date.year) + ',' + str(model_end_date.month) + ',' + str(model_end_date.day) + ',0,0,0'
print(ea)
ia = ' -set var_init_file prms_save_' + initdate.strftime('%Y-%m-%d') + '.init'
print(ia)

arg = 'docker run --rm -v ' + work_dir + ':/work nhmusgs/prms:5.0.0 ' + fa + sa + ea + ia
print(arg)
arg_l = arg.split() # need this for whatever reason; don't move arg.split into the func call below

subprocess.check_output(arg_l)

 -set start_date 2019,3,31,0,0,0
 -set end_date 2019,5,30,0,0,0
 -set var_init_file prms_save_2019-03-30.init
docker run --rm -v /work/markstro/operat/setup/test/NHM-PRMS_CONUS:/work nhmusgs/prms:5.0.0 ./NHM-PRMS.control -set init_vars_from_file 1 -set save_vars_to_file 0 -set start_date 2019,3,31,0,0,0 -set end_date 2019,5,30,0,0,0 -set var_init_file prms_save_2019-03-30.init


b'\n\nWARNING: nhruOut_format is duplicated in the control file ./NHM-PRMS.control.\n\n\n\n\n                         U.S. Geological Survey\n               Precipitation-Runoff Modeling System (PRMS)\n                        Version 5.0.0 05/10/2019\n\n        Process            Available Modules\n--------------------------------------------------------------------\n  Basin Definition: basin\n    Cascading Flow: cascade\n  Time Series Data: obs, water_use_read, dynamic_param_read\n   Potet Solar Rad: soltab\n  Temperature Dist: temp_1sta, temp_laps, temp_dist2, climate_hru\n       Precip Dist: precip_1sta, precip_laps, precip_dist2,\n                    climate_hru\nTemp & Precip Dist: xyz_dist, ide_dist\n    Solar Rad Dist: ccsolrad, ddsolrad, climate_hru\nTranspiration Dist: transp_tindex, climate_hru, transp_frost\n      Potential ET: potet_hamon, potet_jh, potet_pan, climate_hru,\n                    potet_hs, potet_pt, potet_pm, potet_pm_sta\n      Interception: intcp\n     Snow 

Determine if the run completed successfully and evaluate for goodness.

Here's the initial version:
1) Check to see if the file work_dir + output/ + NHM-PRMS.out exists
2) Check for the last line "Execution elapese time..." If PRMS doesn't make it all the way through the time loop, this line won't be there.3) Read the amount of time from the last line. It should be greater than (say) 1 minute. This may need to be refined.

If any of these checks fail, this step should return False; otherwise, return True. This python should go into it's own container as the "ONHM Verifier".

In [48]:
fname = work_dir + '/output/NHM-PRMS.out'
min_time = 1

verified = True
if not os.path.isfile(fname):
    verified = False
    
else:
    with open(fname, 'r') as f:
        found = False
        for line in f:
            if 'Execution elapsed time' in line:
                found = True
                break
                
        if not found:
            verified = False
            
        else:
            tok = line.split()
            mn = int(tok[3])
            sc = float(tok[5])
                
            if mn < min_time:
                verified = False


print(verified)  # this should be the return value of the container


True


If the current model run was good, run prms_outputs2_ncf.py to convert the output to ncf. This will eventually run in a container.

In [29]:
# Not sure about the best way to handle importation of modules in jupyter,
# but this seems to work
# My PRMS python utilities are in onhm-runners/prms_utils

if python_module_path not in sys.path:
    sys.path.append(python_module_path)

# Spell out full paths -- don't use string variables -- don't know why.
%run '/work/markstro/operat/repos/onhm-runners/out2ncf/prms_outputs2_ncf.py' '/work/markstro/operat/setup/test/NHM-PRMS_CONUS/'

setting dir = /work/markstro/operat/setup/test/NHM-PRMS_CONUS/
processing soil_moist


KeyboardInterrupt: 

If the current model run was good, run PRMS-NHM in the workspace to update the init file from whatever date it is at to yesterday.

In [ ]:
os.chdir(work_dir)

# get the list of existing init files.
old_init_file_name_list = []
os.chdir(work_dir)
for file in glob.glob("*.init"):
    old_init_file_name_list.append(file)
    
print(old_init_file_name_list)

# Create the new init file

fa = './NHM-PRMS.control -set init_vars_from_file 1 -set save_vars_to_file 1'

# figure out the following from start_date and end_date
sa = ' -set start_date ' + str(init_start_date.year) + ',' + str(init_start_date.month) + ',' + str(init_start_date.day) + ',0,0,0'
print(sa)
ea = ' -set end_date ' + str(init_end_date.year) + ',' + str(init_end_date.month) + ',' + str(init_end_date.day) + ',0,0,0'
print(ea)

ia = ' -set var_init_file prms_save_' + init_start_date.strftime('%Y-%m-%d') + '.init' + ' -set var_save_file prms_save_' + init_end_date.strftime('%Y-%m-%d') + '.init'
print(ia)

arg = 'docker run --rm -v ' + work_dir + ':/work nhmusgs/prms:5.0.0 ' + fa + sa + ea + ia
arg_l = arg.split() # need this for whatever reason; don't move arg.split into the func call below

subprocess.check_output(arg_l)

Check to make sure that the PRMS init model run worked. If it did, erase the old init files.



Done for this day's step. Wait until tomorrow and go back to the date computation step and repeat subsequent steps. 